## Parsing XML (Extensible Markup Language)

Documentation : https://fr.wikipedia.org/wiki/Extensible_Markup_Language

Librairie lxml : https://lxml.de/tutorial.html

In [ ]:
# imports
import pandas as pd
from lxml import etree

# options d'affichage
pd.set_option("display.min_rows", 16)

**Exemple**

Les 150 propositions de la Convention Citoyenne pour le Climat

https://www.data.gouv.fr/fr/datasets/les-150-propositions-de-la-convention-citoyenne-pour-le-climat/

In [ ]:
# visualisation du fichier
%pycat propositions.xml

In [ ]:
# parsing XML
root = etree.parse('propositions.xml')

In [ ]:
# type
type(root)

In [ ]:
# trouver un élément
element = root.find('categorie')

In [ ]:
# type
type(element)

In [ ]:
# parent
element.getparent()

In [ ]:
# attribut
element.attrib['titre']

In [ ]:
# trouver un élément
element = root.find('categorie').find('sousCategorie').find('proposition')

In [ ]:
# text
element.text

In [ ]:
# nombre de catégories: getiterator
len([node for node in root.getiterator('categorie')])

In [ ]:
# nombre de sous-catégories: getiterator
len([node for node in root.getiterator('sousCategorie')])

In [ ]:
# nombre de propositions: getiterator
len([node for node in root.getiterator('proposition')])

In [ ]:
# print categorie / sousCategorie / proposition
for node_categorie in root.getiterator('categorie'):
    
    print(node_categorie.attrib['titre'])
    
    for node_sousCategorie in node_categorie.getiterator('sousCategorie'):
        
        print('> ', node_sousCategorie.attrib['titre'])
        
        for node_proposition in node_sousCategorie.getiterator('proposition'):
            
            print('>> ', node_proposition.text)

In [ ]:
# imports
import pandas as pd

# fabrication d'un DataFrame à partir d'une liste de dict
l = [
    {'a': 1, 'b': 2},
    {'a': 3, 'b': 4},
    {'a': 5, 'b': 6},
]

pd.DataFrame(l)

In [ ]:
# fabrication d'un DataFrame à partir du XML
liste = [
    {'categorie': node_category.attrib['titre'],
    'sousCategorie': node_subcategory.attrib['titre'],
    'proposition': node_proposition.text}
        for node_category in root.getiterator('categorie')
            for node_subcategory in node_category.getiterator('sousCategorie')
                for node_proposition in node_subcategory.getiterator('proposition')]

df = pd.DataFrame(liste)
df

In [ ]:
# catégories
df['categorie'].value_counts()

In [ ]:
# requêtes
df['proposition'].str.contains('biodiversité').sum()

In [ ]:
# requêtes
print(*df.loc[df['proposition'].str.contains('biodiversité'), 'proposition'].values, sep='\n')

In [ ]:
# requêtes
df['proposition'].str.contains('énergie').sum()

In [ ]:
# requêtes
print(*df.loc[df['proposition'].str.contains('énergie'), 'proposition'].values, sep='\n')

**Exercice 1**

Produire un DataFrame avec les colonnes : categorie, sousCategorie, oui (float).

In [ ]:
# fabrication d'un DataFrame à partir du XML
liste = [
    {'categorie': node_category.attrib['titre'],
    'sousCategorie': node_subcategory.attrib['titre'],
    'oui': node_oui.text    
    }
        for node_category in root.getiterator('categorie')
            for node_subcategory in node_category.getiterator('sousCategorie')
                for node_oui in node_subcategory.getiterator('oui')]

df = pd.DataFrame(liste)
df["oui"]=df["oui"].str.replace(',','.',1).astype(float)
df.head()

**Exercice 2**

- Produire un DataFrame avec les colonnes : categorie, sousCategorie, oui (float), non (float), blancs (float).
- Calculer les sommes oui + non + blancs et oui + non &#9786;

In [ ]:
# fabrication d'un DataFrame à partir du XML
liste = [
    {'categorie': node_category.attrib['titre'],
     'sousCategorie': node_subcategory.attrib['titre'],
     'oui': node_vote.find("oui").text,
     'non': node_vote.find("non").text,
     'blancs': node_vote.find("blancs").text
    }
        for node_category in root.getiterator('categorie')
            for node_subcategory in node_category.getiterator('sousCategorie')
                for node_vote in node_subcategory.getiterator('vote')
]

df = pd.DataFrame(liste)
df["oui"]=df["oui"].str.replace(',','.',1).astype(float)
df[["non","blancs"]]=df[["non","blancs"]]\
    .applymap(lambda x : x.replace(',','.',1)).astype(float)
df

In [ ]:
df[["oui","non"]].sum(axis=1)

**Exercice 3**

- Calculer un dictionnaire fréquentiel des mots  des propositions.
- Le mettre dans un objet de type Series.

In [ ]:
# re.findall()
import re
re.findall('[A-Za-zÀ-ÿ0-9]+', 'la fonction findall est très utile.')

In [ ]:
from collections import Counter
c = Counter()
df_prop["proposition"].apply(lambda x: c.update(re.findall('[A-Za-zÀ-ÿ0-9]+',x.lower())))
voc = pd.Series(dict(c))
voc = voc.sort_values(ascending=False)
voc #stopword > nltk.corpus stopwords

In [ ]:
# Counter
from collections import Counter
x = Counter()
#x.update(a)
#[(l,k) for k,l in sorted([(j,i) for i,j in x.items()], reverse=True)]

#### Validation XML / XSD

Il existe un langage de description de schéma XML appelé XSD (XML Schema Definition). Un fichier XML peut être écrit selon un schéma XSD particulier. Il existe une librairie Python qui peut vérifier qu'un fichier XML est valide selon un schéma XSD donné.

Librairie xmlschema :
https://pypi.org/project/xmlschema/

L'utilisation de la librairie est assez simple :

<pre>
>>> # vérification que le fichier "file.xml" est valide dans le schéma "schema.xsd"
>>> import xmlschema
>>> my_schema = xmlschema.XMLSchema('schema.xsd')
>>> #
>>> # retourne un booléen selon la validité
>>> my_schema.is_valid('file.xml')
</pre>